# To check https://stackoverflow.com/questions/42081257/keras-binary-crossentropy-vs-categorical-crossentropy-performance

In [15]:
import json
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import SGD, RMSprop, Adagrad

from keras.applications.vgg16 import VGG16

import balloonml
import cv2

import importlib
importlib.reload(balloonml)

<module 'balloonml' from '/home/wouter/notebooks/disdat-ml/keras/balloonml.py'>

## Defining training parameters

In [13]:
config ={}


config['train_data_dir'] = '../clean-data'
config['validation_data_dir'] = '../clean-validation'

config['bottleneck_train'] = 'bottleneck_features_train.npy'
config['bottleneck_validation'] = 'bottleneck_features_validation.npy'

# True if bottleneck features are not available yet, 
# False if they are and can be lodaded
calculate_bottleneck = False 

config['epochs'] = 25
config['epochs-refine'] = 20

config['batch_size'] = 32

lrs = [0.002, 0.002, 0.001, 0.001, 0.0005, 0.0005]
decays = [0,1,0,1,0,1]

## Saving or loading bottleneck features

In [3]:
if calculate_bottleneck:
    model = VGG16(include_top=False, weights='imagenet')
    train_data = balloonml.bottleneck(model,config['train_data_dir'],config['bottleneck_train'], config['batch_size'])
    validation_data = balloonml.bottleneck(model,config['validation_data_dir'],config['bottleneck_validation'], config['batch_size'])
else:
    train_data = np.load(config['bottleneck_train'])
    validation_data = np.load(config['bottleneck_validation'])

In [14]:
for lr, decay in zip(lrs, decays):
    config['optimizer']='sgd'
    config['lr']=lr
    config['decay']=decay * (lr)/(100000/config['batch_size'])
    config['output']='disdat-104-v5-softmax'+'-lr'+str(lr)+'-decay'+str(decay)

    with open(config['output']+'.config', 'w') as fp:
        json.dump(config, fp)
        
    if config['optimizer']=='sgd':
        optimizer=SGD(lr=config['lr'], decay=config['decay'])
    elif config['optimizer']=='rmsprop':
        optimizer=RMSprop(lr=config['lr'], decay=config['decay'])
    elif config['optimizer']=='adagrad':
        optimizer=Adagrad(lr=lr, epsilon=1e-08, decay=0.0)
    history, model = balloonml.train_top(
        train_data, 
        validation_data,
        train_data_dir=config['train_data_dir'], 
        validation_data_dir=config['validation_data_dir'],
        optimizer=optimizer,
        batch_size = config['batch_size'],
        epochs=config['epochs'],
        output=config['output'])

Found 115042 images belonging to 104 classes.
Found 28707 images belonging to 104 classes.


ValueError: Error when checking target: expected dense_4 to have shape (None, 1) but got array with shape (115042, 104)

## Running model

In [19]:
config['output']='disdat-104-v5-softmax-lr0.001-decay0'

## Refining model

In [ ]:
refine_optimizer=SGD(lr=2e-4, momentum=0.9)

refined_history, refined_model = balloonml.finetune(
    train_data_dir=config['train_data_dir'],
    validation_data_dir=config['validation_data_dir'], 
    optimizer=refine_optimizer, 
    weights_top_layer=config['output']+'.h5', 
    batch_size=config['batch_size'], 
    epochs=config['epochs-refine'],
    output=config['output']+'-refined')
balloonml.plotResult(refined_history)

Found 115042 images belonging to 104 classes.
Found 115042 images belonging to 104 classes.
Found 28707 images belonging to 104 classes.
Epoch 1/20
 265/3595 [=>............................] - ETA: 3466s - loss: 2.0884 - acc: 0.4823

In [ ]:
probabilities = model.predict(image)
print(classes)
print(probabilities)